In [1]:
@file:DependsOn("ai.timefold.solver:timefold-solver-core:1.13.0")
@file:DependsOn("com.fasterxml.jackson.module:jackson-module-kotlin:2.17.1")
@file:DependsOn("org.jetbrains.kotlinx:kotlinx-serialization-json:1.5.1")
@file:DependsOn("org.slf4j:slf4j-simple:2.0.7") // For logging


In [2]:
import com.fasterxml.jackson.annotation.JsonFormat
import com.fasterxml.jackson.module.kotlin.jacksonObjectMapper
import com.fasterxml.jackson.module.kotlin.readValue
import ai.timefold.solver.core.api.domain.solution.PlanningSolution
import ai.timefold.solver.core.api.domain.solution.PlanningScore
import ai.timefold.solver.core.api.domain.variable.PlanningListVariable
import ai.timefold.solver.core.api.score.buildin.hardsoftlong.HardSoftLongScore
import ai.timefold.solver.core.api.score.stream.Constraint
import ai.timefold.solver.core.api.score.stream.ConstraintFactory
import ai.timefold.solver.core.api.score.stream.ConstraintProvider
import ai.timefold.solver.core.api.score.stream.ConstraintCollectors
import ai.timefold.solver.core.api.solver.SolverFactory
import ai.timefold.solver.core.api.solver.Solver
import java.io.File
import java.time.Duration
import java.time.LocalTime
import org.slf4j.LoggerFactory


In [3]:
data class Location(
    val latitude: Double,
    val longitude: Double
) {
    fun distanceTo(other: Location): Double {
        val earthRadius = 6371e3 // Earth radius in meters
        val lat1 = Math.toRadians(latitude)
        val lat2 = Math.toRadians(other.latitude)
        val deltaLat = Math.toRadians(other.latitude - latitude)
        val deltaLon = Math.toRadians(other.longitude - longitude)

        val a = Math.sin(deltaLat / 2) * Math.sin(deltaLat / 2) +
                Math.cos(lat1) * Math.cos(lat2) *
                Math.sin(deltaLon / 2) * Math.sin(deltaLon / 2)
        val c = 2 * Math.atan2(Math.sqrt(a), Math.sqrt(1 - a))

        val distance = earthRadius * c // Distance in meters
        return distance
    }
}


In [4]:
import com.fasterxml.jackson.annotation.JsonProperty
import ai.timefold.solver.core.api.domain.variable.InverseRelationShadowVariable
import ai.timefold.solver.core.api.domain.variable.NextElementShadowVariable
import ai.timefold.solver.core.api.domain.variable.PreviousElementShadowVariable
import ai.timefold.solver.core.api.domain.lookup.PlanningId
import java.time.LocalTime
import java.time.format.DateTimeFormatter

// Define Client class (Problem Fact, not a Planning Entity)
data class Client(
    @JsonProperty("Client_ID")
    val clientId: String,

    @JsonProperty("Need")
    val need: String, // "Waste", "Enrobé", "Grue"

    @JsonProperty("Volume_t")
    val volumeT: Double,

    @JsonProperty("Volume_kg")
    val volumeKg: Double,

    @JsonProperty("Access_Difficulty")
    val accessDifficulty: String, // "Yes" or "No"

    @JsonProperty("Time_Window_Label")
    val timeWindowLabel: String, // "Morning", "Afternoon", "All Day"

    @JsonProperty("Time_Window_Start")
    val timeWindowStart: String, // "08:00", "13:00", etc.

    @JsonProperty("Time_Window_End")
    val timeWindowEnd: String,

    @JsonProperty("Latitude")
    val latitude: Double,

    @JsonProperty("Longitude")
    val longitude: Double
) {
    val location: Location = Location(latitude, longitude)

    // Convert "Yes"/"No" to Boolean
    val hasAccessDifficulty: Boolean
        get() = accessDifficulty.equals("Yes", ignoreCase = true)

    // Time windows as LocalTime
    val timeWindowStartTime: LocalTime
        get() = LocalTime.parse(timeWindowStart, DateTimeFormatter.ofPattern("HH:mm"))

    val timeWindowEndTime: LocalTime
        get() = LocalTime.parse(timeWindowEnd, DateTimeFormatter.ofPattern("HH:mm"))

    override fun toString(): String = clientId
}

// Define Truck class
@PlanningEntity
data class Truck(
    @PlanningId
    val truckId: String,

    val truckType: String, // e.g., "Multilift 4 essieux", "Multilift 4 essieux Tridem", "Camion-grue 4 essieux"

    val capacityKg: Double,

    val equipment: String, // e.g., "Benne 10 m3", "Benne 20 m3", "Pont 13 m3", "Thermosilo"

    val canAccessDifficultLocations: Boolean,

    val homeLocation: Location // Warehouse location
) {
    @PlanningListVariable(valueRangeProviderRefs = ["clientRange"], nullable = true) // Allow unassigned clients
    var assignedClients: MutableList<Client> = mutableListOf()

    // No-arg constructor for Timefold
    constructor() : this(
        truckId = "",
        truckType = "",
        capacityKg = 0.0,
        equipment = "",
        canAccessDifficultLocations = false,
        homeLocation = Location(0.0, 0.0)
    )

    override fun toString(): String = truckId
}


Line_4.jupyter.kts (58:2 - 16) Unresolved reference: PlanningEntity
Line_4.jupyter.kts (73:69 - 77) Cannot find a parameter with this name: nullable